<a href="https://colab.research.google.com/github/robgon-art/DeepHaiku/blob/main/Deep_Haiku_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Haiku Generator**
## Generating rhythmic prose after finetuning a large transformer with phonemes

By Robert. A Gonsalves</br>

![image](https://raw.githubusercontent.com/robgon-art/DeepHaiku/main/deep_haiku.jpg)

You can see my article on Medium.

The source code and generated images are released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/).</br>
![CC BYC-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)

## Acknowledgements
- M. Grootendorst, KeyBERT: Minimal keyword extraction with BERT (2020)
- W. Zhu and S. Bhat, GRUEN for Evaluating Linguistic Quality of Generated Text (2020)
- M. Bernard, Phonemizer: Text to Phones Transcription for Multiple Languages in Python (2016)
- GPT-J, Mesh-Transformer-JAX: Model-Parallel Implementation of Transformer Language Model with JAX (2021)
- R. Caruana, Multitask learning (1997)
- E. Hu, et al., LoRA: Low-rank Adaptation of Large Language Models (2021)
- L. Hanu and the Unitary Team, Detoxify (2020)
- Trained on Haikus collected by [bfbarry](https://www.kaggle.com/bfbarry/haiku-dataset) and [Harshit Jhalani](https://www.kaggle.com/hjhalani30/haiku-dataset) on Kaggle.com

In [ ]:
#@title Initialize the System

!pip install --upgrade --no-cache-dir gdown
!git clone https://github.com/unitaryai/detoxify
!pip install transformers==4.16.2
!pip install bitsandbytes-cuda111
!git clone https://github.com/robgon-art/GRUEN
!pip install wmd
!pip install --upgrade --no-cache-dir gdown
!gdown --id 1S-l0L_YOzn5KhYHdB8iS37qKwuUhHP0G
!gdown --id 10LpkO5Vm_zOu723FVk6cCeRsv_qyYLdL
!unzip cola_model.zip
!pip install phonemizer
!sudo apt-get install festival

import transformers
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from tqdm.auto import tqdm
from phonemizer import phonemize
from phonemizer.separator import Separator
import nltk
nltk.download('punkt')
import sys
sys.path.append("GRUEN")
import GRUEN.Main as gruen
import sys
sys.path.append("/content/detoxify")
from detoxify import Detoxify

def get_festival_phonemes(line):
  phn = phonemize(line, language='en-us', backend='festival', with_stress=False,
      separator=Separator(phone=None, word=' ', syllable="|"), strip=True)
  return phn

text = ["pet pug arthur"]
print(get_festival_phonemes(text))

def count_syllables(doc):
  phrases = doc.split("/")
  counts = []
  for p in phrases:
    count = 0
    words = p.split(" ")
    for w in words:
      syllables = w.split("|")
      count += len(syllables)
    counts.append(count)
  return counts

print(count_syllables("waa|shihng dhax dih|shaxz/thihng|kaxng ax|bawt ax hhay|kuw/werdz flow layk wao|ter"))

doc = ["Autumn is nearing. Golden trees tempting my eyes. While songs tease my ears"]
gruen_results = gruen.get_gruen(doc)
print("gruen: ", gruen_results)

from detoxify import Detoxify
results = Detoxify('original').predict("Autumn is nearing / Golden trees tempting my eyes / While songs tease my ears")
for key, value in results.items():
  print(key, ' : ', round(value, 5))

class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

import transformers
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

gpt = torch.load("/content/gpt-j-8bit_deep_haikul.pt",  map_location=torch.device('cuda'))
gpt.eval()

In [14]:
#@title Generate Haikus

import warnings
warnings.filterwarnings("ignore")

topic = 'autumn' #@param {type:"string"}
only_show_5_7_5 = True #@param {type:"boolean"}
quality_filter = 0.45 #@param {type:"slider", min:0, max:1, step:0.01}
toxicity_filter = 0.85 #@param {type:"slider", min:0, max:1, step:0.01}

prompt = "(" + topic.strip()
if not "=" in topic:
  prompt += " ="
# print("'" + prompt + "'")
with torch.no_grad():
  prompt_tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
  sample_outputs = gpt.generate(prompt_tokens, max_length=40, do_sample=True, 
    num_return_sequences=20, temperature=0.8)
    
haikus = []
haikus_no_slashes = []

for i, sample_output in enumerate(sample_outputs):
  doc = (tokenizer.decode(sample_outputs[i], skip_special_tokens=True))
  parts = doc.split(")")
  haiku_with_prompt = parts[0][1:].strip()
  haiku = haiku_with_prompt.split(" = ")[1].strip()
  haikus.append(haiku)
  haikus_no_slashes.append(haiku.replace(" / ", " "))

quality = gruen.get_gruen(haikus_no_slashes)

print()
heading = "Deep Haiku Generation for " + topic
print(heading.ljust(90) + "Syllables".ljust(12) + "Quality".ljust(10) + "Toxicity")

import re
for h, q in zip(haikus, quality):
  hp = []
  parts = h.split("/")
  for part in parts:
    part = part.strip()
    part = re.sub(r"[^A-Za-z| ]+", "", part)
    p = get_festival_phonemes(part)
    hp.append(p)
  phones = "/".join(hp)
  syllables = count_syllables(phones)
  if not only_show_5_7_5 or syllables == [5, 7, 5]:
    results = Detoxify('original').predict(h)
    t = results["toxicity"]
    # print(h, syllables, s, t)
    # if True:
    if q > quality_filter and t < toxicity_filter: 
      h = h.replace(". / ", " / ")
      if h.endswith("."):
        h = h[:-1]
      print(h.ljust(90) + str(syllables).ljust(12) + str(round(q, 5)).ljust(10) + 
            str(round(t,5)))
      # print(h)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Evaluating: 100%|██████████| 59/59 [00:00<00:00, 65.42it/s]



Deep Haiku Generation for autumn                                                          Syllables   Quality   Toxicity
Autumn arrives soon / Without warning once again / I stare at the trees                   [5, 7, 5]   0.75344   0.00078
Autumn air fills the / House with salty, sweet candy / Apples, cinnamon                   [5, 7, 5]   0.56015   0.00062
Autumn winds arrive / Parading through the tree tops / Leaf colors flutter                [5, 7, 5]   0.78927   0.0006
Autumn air, cool breeze / Candy colors painted skies / Fall is near, but come             [5, 7, 5]   0.69261   0.00062
Autumn winds arrive / Parading through the walnut / Panic in the hills                    [5, 7, 5]   0.6862    0.00073
Autumn Equinox / Where the sun and moon are in? /balance this night is                    [5, 7, 5]   0.60462   0.00073
I think it's okay / To eat a little bit of / Candy in autumn                              [5, 7, 5]   0.57373   0.00067
Autumn colours here / Rainy days are be